In [1]:
import pandas as pd


In [2]:
features = 6
fname = 'results' + str(features) + 'parsed.csv'

feature_importance  = {}
feature_frequency = {}

df = pd.read_csv(fname)

ca = df['RFW CA']
features = df['RFW AUC Features']

for acc, f in zip(ca, features):
    for fp in f.split(','):
        fp = fp.strip()
        
        if fp not in feature_importance:
            feature_importance[fp] = (1 - acc)**2
            feature_frequency[fp] = 1
        else:
            feature_importance[fp] += (1 - acc)**2
            feature_frequency[fp] += 1

for k in feature_importance:
    feature_importance[k] = feature_importance[k] / feature_frequency[k]
print(            
sorted(feature_importance.items(), key=lambda x : x[1])
    )

[('nof_Rings', 0.07289999999999995), ('molVolume', 0.0730921952160494), ('nof_HBA', 0.07396389045559942), ('molPSA', 0.0741820023148148), ('C_R0', 0.07497305073302471), ('molLogS', 0.07594901284889963), ('nof_OH', 0.07688751908706948), ('nof_NH2', 0.07900340183019275), ('molLogP', 0.08437430023414215), ('PSA/Area', 0.08607921767653288), ('posCharge/Volume', 0.08788548503672451), ('nof_posCharge', 0.08999150303320558), ('C_sp3', 0.09157964506172836), ('nof_PO4', 0.0917159185862692), ('nof_COOH', 0.09342012701804371), ('nof_HBD', 0.09470423456790121), ('C_R1', 0.09474924768518522), ('nof_SO3H', 0.09561088027803714), ('negCharge/Volume', 0.0967719749367842), ('nof_negCharge', 0.09718152419532629), ('nof_SH', 0.09927573812019566), ('C_R2', 0.10015100546058878), ('Complexity', 0.10207509920634916), ('nof_acetyl', 0.10411794410150893)]
